In [1]:
import utils

from train_2 import EditTrainer
from config import EditConfig


In [3]:
model, tokenizer = utils.loadOTSModel()
dataloader = utils.retrieveDataloader(
        tokenizer, 
        bs=1, 
        dataset='train'
    )
# del model

In [4]:

class EditConfig:
    def __init__(self):
        self.inner_lr = 1e-3
        self.outer_lr = 1e-5
        self.epochs = 1
        self.max_training_samps = 1e4
        self.n_edit_steps = 1
        self.cedit = 1
        self.cloc = 1

        self.debug = True
        self.model_save_pt = 2500
        self.model_dir = '../models/finetune'


## Test Run

In [6]:
%load_ext autoreload
%autoreload 2
import utils

from train_2 import EditTrainer
from config import EditConfig


trainer = EditTrainer(EditConfig(), dataloader)
trainer.run()

Starting Training
('Epoch: 0; TrainStep 0; ', 'l_base 5.140927314758301; l_edit 9.583337783813477; l_loc 0.0791158452630043; total 14.803380966186523')
('Epoch: 0; TrainStep 1; ', 'l_base 4.468011856079102; l_edit 8.450944900512695; l_loc 0.2951970100402832; total 13.214153289794922')
('Epoch: 0; TrainStep 2; ', 'l_base 3.9141573905944824; l_edit 9.361299514770508; l_loc 0.1888570338487625; total 13.464313507080078')
('Epoch: 0; TrainStep 3; ', 'l_base 6.0544114112854; l_edit 7.479859828948975; l_loc 0.09249913692474365; total 13.62677001953125')
('Epoch: 0; TrainStep 4; ', 'l_base 4.703126907348633; l_edit 5.6159210205078125; l_loc 0.2222064882516861; total 10.541254043579102')
('Epoch: 0; TrainStep 5; ', 'l_base 3.9404916763305664; l_edit 8.285829544067383; l_loc 0.10465022921562195; total 12.330970764160156')
('Epoch: 0; TrainStep 6; ', 'l_base 4.77923059463501; l_edit 6.186927795410156; l_loc 0.05019443482160568; total 11.016352653503418')
('Epoch: 0; TrainStep 7; ', 'l_base 3.9934

KeyboardInterrupt: 

## Sample from own logits

In [9]:
import torch

In [5]:
# text = "Poppies and lupines are the primary flowers that dominate the green, lush rolling meadows in the spring to early summer, but in the late summer the hills turn gold and are covered with flowers such as the cluster-lily, gumweed, Mules ear, and farewell to spring. In shaded areas, one can expect to see thimbleberry and a variety of ferns growing. "

In [65]:
text = "Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made it particularly popular as a means of adding programmable interfaces to existing applications. Van Rossum's vision of a small core language with a large standard library and easily extensible interpreter stemmed from his frustrations with ABC, which espoused the opposite approach."

In [66]:
token_list = tokenizer.encode(text)
token_out = tokenizer(
    text
)
tokens, mask = map(
    torch.tensor, 
    [token_out['input_ids'], token_out['attention_mask']]
)

In [67]:
tokens.size()

torch.Size([79])

In [75]:
feed = tokens[:40]
print(tokenizer.decode(token_list[:40]))

Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made it particularly popular as a means of adding programmable


In [89]:

inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt")
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [83]:
generation_output

GreedySearchDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  3290,   318, 13779,   290,   220, 17479,    13,
           314,  1842,   683,   523,   881,    13,   314,  1842,   683,   523]]), scores=(tensor([[-43.4955, -46.0844, -46.9108,  ..., -57.7377, -51.5319, -48.3616]]), tensor([[-48.8799, -50.3510, -55.1450,  ..., -64.8533, -62.6215, -52.4255]]), tensor([[-67.7690, -65.9575, -67.5762,  ..., -76.5747, -74.8564, -60.7996]]), tensor([[-86.5862, -85.9131, -89.9283,  ..., -97.0860, -86.9367, -87.2296]]), tensor([[-66.0425, -67.5343, -71.4601,  ..., -75.1458, -72.8039, -67.9962]]), tensor([[-59.0425, -63.4027, -69.0216,  ..., -77.3129, -73.8313, -64.7133]]), tensor([[-60.0665, -62.8150, -68.4672,  ..., -72.5271, -70.3083, -63.8665]]), tensor([[-47.8266, -52.4544, -58.7991,  ..., -65.4726, -64.2974, -52.0334]]), tensor([[-71.7299, -70.1800, -72.2610,  ..., -82.0179, -80.4628, -63.5811]]), tensor([[-86.2451, -85.7662, -90.2981,  ..., -97.1315, -86.9278, -86.6646]]), tensor([[-6

In [99]:
tokenizer.decode(tokenizer.eos_token_id)

'<|endoftext|>'

In [103]:
text = "Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made"

In [107]:
prompt_text = text
encoded_prompt = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
input_ids = encoded_prompt

output_sequences = model.generate(
    input_ids=input_ids,
    max_length=500 + len(encoded_prompt[0]),
    temperature=1.2,
#     top_k=args.k,
#     top_p=args.p,
    repetition_penalty=1,
    do_sample=True,
    num_return_sequences=5,
)

# Remove the batch dimension when returning multiple sequences
if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

generated_sequences = []

for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find('<|endoftext|>')]

    # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
    total_sequence = (
        prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
    )

    generated_sequences.append(total_sequence)
    print(total_sequence)

generated_sequences

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== GENERATED SEQUENCE 1 ===
Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made the most sense for many companies at our top level and enabled us greatly in recent years. But if there are a problem associated with such work-in other ways it is only going through individual projects running on any distribution where pip might not offer much option. Python 3 is no exception; by working from a separate source code base Python has been easy to set up and maintain to the point it takes us a full year. A version of Python 3 has since been adopted (so far, it has beeved almost 20%).
=== GENERATED SEQUENCE 2 ===
Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made the most sense for many companies at our top level and enabled us greatly in recent years. But if there are a problem assoc

['Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made the most sense for many companies at our top level and enabled us greatly in recent years. But if there are a problem associated with such work-in other ways it is only going through individual projects running on any distribution where pip might not offer much option. Python 3 is no exception; by working from a separate source code base Python has been easy to set up and maintain to the point it takes us a full year. A version of Python 3 has since been adopted (so far, it has beeved almost 20%).',
 "Rather than having all of its functionality built into its core, Python was designed to be highly extensible (with modules). This compact modularity has made the most sense for many companies at our top level and enabled us greatly in recent years. But if there are a problem associated with such work-in other ways it is only going 